# Language Model
> Modeling language on wikitext data.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/chart-preview.png

```
# "My Title"
> "Language Model"

- toc: true
- branch: master
- badges: true
- comments: true
- author: Hamel Husain & Jeremy Howard
- categories: [fastpages, jupyter]
```

In [3]:
import transformers
from datasets import load_dataset
datasets = load_dataset("wikitext", "wikitext-2-raw-v1")

Downloading: 8.33kB [00:00, 1.99MB/s]                   
Downloading: 5.83kB [00:00, 1.79MB/s]                   


Downloading: 100%|██████████| 4.72M/4.72M [00:01<00:00, 4.49MB/s]
0 examples [00:00, ? examples/s]2021-11-24 13:20:29.789737: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
                                

Dataset wikitext downloaded and prepared to /home/aswin/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20. Subsequent calls will reuse this data.


In [4]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [8]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
def show_random_elements(dataset, n=5):
    picks = []
    for i in range(n):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(datasets["train"])

,text
0,"NY 38A ( 21 @.@ 91 miles or 35 @.@ 26 kilometres ) runs from Moravia to NY 359 , near Mandana and NY 41A in southwestern Onondaga County , and then towards Auburn . It was assigned as part of the 1930 renumbering of state highways in New York . \n"
1,"Since treatment options for relapsed AML are so limited , palliative care or enrolment in a clinical trial may be offered . \n"
2,""" Chapter 1 "" sets the tone for the environs of the series . According to David Carr the political environs have such "" marbleness "" that it belies the clandestine nature of political activities , including those of Underwood who says he is there to "" clear the pipes and keep the sludge moving "" . The fictional newspaper , The Washington Herald , is set with "" brutal "" lighting and drab furniture , in part because it was filmed at the real life Baltimore Sun offices . Carr uses several pejorative adjectives to describe Barnes ' apartment including sad , grubby , dirty , dreary and humble but note that this implies that the digital revolution is dominated by people "" on laptops who have no furniture "" . Similarly , Underwood and his associates are nattily clad , Barnes shows a lack of fashion recognition . \n"
3,= = Record = = \n
4,"It had a 13 @,@ 000 @-@ square @-@ foot ( 1 @,@ 200 m2 ) gallery for art , history , and science exhibits . It also had a 280 @-@ seat indoor theater , a 500 @-@ seat outdoor theater , classrooms , an 80 @-@ seat demonstration kitchen , a rare book library , a wine @-@ tasting area , a café ( named American Market Cafe ) , gift shop ( named Cornucopia ) , and 3 @.@ 5 acres ( 1 @.@ 4 ha ) of landscaped edible gardens . The building 's architect was Polshek Partnership Architects . Julia 's Kitchen was a restaurant inside the Copia building that focused on seasonal dishes and was named for honorary trustee Julia Child , who loaned part of her kitchen to the restaurant , a wall of 49 pans , pots , fish molds , and other tools and objects . Within a year of the center 's closing , the items were sent to the Smithsonian Institution 's National Museum of American History , where they are included in the Julia Child 's kitchen exhibit , which up until that point was only missing that portion . The restaurant had a 1 @,@ 700 @-@ square @-@ foot ( 160 m2 ) dining room ( for 180 seats ) , an outdoor seating area ( 4 @,@ 300 square feet ( 400 m2 ) ) and a 2 @,@ 500 @-@ square @-@ foot ( 230 m2 ) kitchen . The gardens had fruit orchards , a pavilion with a kitchen and large dining table , and a small vineyard with 60 vines and 30 different grape varieties . The restaurant and café were both operated by local caterer Seasoned Elements , and later Patina Restaurant Group . \n"


In [9]:
model_checkpoint = "gpt2"
tokenizer_checkpoint = "sgugger/gpt2-like-tokenizer"

In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

Downloading: 100%|██████████| 236/236 [00:00<00:00, 43.5kB/s]
Downloading: 100%|██████████| 387k/387k [00:00<00:00, 735kB/s]
Downloading: 100%|██████████| 226k/226k [00:00<00:00, 635kB/s]
Downloading: 100%|██████████| 663k/663k [00:00<00:00, 1.21MB/s]
Downloading: 100%|██████████| 90.0/90.0 [00:00<00:00, 57.8kB/s]


In [11]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [13]:
tokenized_dataset = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/2 [00:00<?, ?ba/s]




#1: 100%|██████████| 2/2 [00:00<00:00,  2.53ba/s]


#0:   0%|          | 0/10 [00:00<?, ?ba/s]


#0:  10%|█         | 1/10 [00:00<00:06,  1.32ba/s]


#0:  20%|██        | 2/10 [00:01<00:05,  1.34ba/s]


#0:  30%|███       | 3/10 [00:02<00:05,  1.34ba/s]



#0:  40%|████      | 4/10 [00:02<00:04,  1.33ba/s]

#0:  50%|█████     | 5/10 [00:03<00:03,  1.41ba/s]



#0:  60%|██████    | 6/10 [00:04<00:02,  1.52ba/s]


#0:  70%|███████   | 7/10 [00:04<00:01,  1.77ba/s]


#0:  80%|████████  | 8/10 [00:04<00:00,  2.00ba/s]


#0:  90%|█████████ | 9/10 [00:05<00:00,  2.18ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]


#0: 100%|██████████| 1/1 [00:00<00:00,  2.18ba/s]


#3: 100%|██████████| 1/1 [00:00<00:00,  1.71ba/s]

#2: 100%|██████████| 1/1 [00:00<00:00,  1.37ba/s]


In [14]:
tokenized_dataset["train"][1]

{'attention_mask': [1, 1, 1, 1, 1, 1],
 'input_ids': [238, 8576, 9441, 2987, 238, 252]}

In [15]:
block_size = 128
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result= {k: [t[i: i + block_size] for i in range(0, total_length, block_size)] for k, t in concatenated_examples.items()}
    result["labels"] = result["input_ids"].copy()
    return result

In [17]:
lm_datasets = tokenized_dataset.map(group_texts, batched=True, batch_size=1000, num_proc=4)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]


#1: 100%|██████████| 2/2 [00:01<00:00,  1.55ba/s]


#0: 100%|██████████| 2/2 [00:01<00:00,  1.33ba/s]


#3: 100%|██████████| 2/2 [00:01<00:00,  1.31ba/s]

#0:   0%|          | 0/10 [00:00<?, ?ba/s]


#0:  10%|█         | 1/10 [00:00<00:08,  1.11ba/s]


#0:  20%|██        | 2/10 [00:01<00:07,  1.05ba/s]


#0:  30%|███       | 3/10 [00:02<00:06,  1.06ba/s]


#0:  40%|████      | 4/10 [00:03<00:05,  1.06ba/s]


#0:  50%|█████     | 5/10 [00:04<00:04,  1.10ba/s]


#0:  60%|██████    | 6/10 [00:05<00:03,  1.14ba/s]



#0:  70%|███████   | 7/10 [00:06<00:03,  1.01s/ba]


#2: 100%|██████████| 10/10 [00:08<00:00,  1.23ba/s]


#0: 100%|██████████| 10/10 [00:08<00:00,  1.14ba/s]


#0:   0%|          | 0/1 [00:00<?, ?ba/s]


#0: 100%|██████████| 1/1 [00:00<00:00,  1.96ba/s]

#1: 100%|██████████| 1/1 [00:00<00:00,  1.34ba/s]


#3: 100%|██████████| 1/1 [00:00<00:00,  1.31ba/s]


In [18]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Ozawa. A large'